# Data management
- In questo notebook procederemo a preparare i dati da utilizzare nel main

# Gestione del dataset delle città
- Dopo aver aggiornato il dataset delle città con i dati fino al 2020-12-01, si provvede alla sistemazione del dataset principale e all'unione di quest'utimo con il dataset aggiornato

In [1]:
# importo pacchetti utili
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
# importo il dataset delle città
df_cities= pd.read_csv('Data/GlobalLandTemperaturesByCity.csv')

In [3]:
df_cities

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
...,...,...,...,...,...,...,...
8599207,2013-05-01,11.464,0.236,Zwolle,Netherlands,52.24N,5.26E
8599208,2013-06-01,15.043,0.261,Zwolle,Netherlands,52.24N,5.26E
8599209,2013-07-01,18.775,0.193,Zwolle,Netherlands,52.24N,5.26E
8599210,2013-08-01,18.025,0.298,Zwolle,Netherlands,52.24N,5.26E


In [75]:
# si calcola la prima data utile per ogni città per capire da dove far partire il dataset 
first_occurencies_cities = {}

for i in range(df_cities.shape[0]):
    
    if df_cities['City'][i] not in first_occurencies_cities.keys():
        
        first_occurencies_cities[df_cities['City'][i]] = df_cities['dt'][i]

In [76]:
# si trova la città che ha la prima osservazione massima 
max_value = max(first_occurencies_cities.values())
max_keys = [k for k, v in first_occurencies_cities.items() if v == max_value]
print(max_keys, max_value)

['Port Moresby'] 1882-01-01


In [77]:
# si eliminano le le osservazioni inferiori alla prima osservazione massima e le osservazioni relative al 2013, 
# che verranno aggiunte in seguito

df_cities= df_cities[(df_cities['dt']>='1882-01-01')&(df_cities['dt']<='2012-12-01')]

In [78]:
# si esaminano le osservazioni mancanti che sono ancora 6640, 
# per alcune città le misure mancanti ravvicinate sono troppe per essere interpolate 
# e quindi si otterrebbero valori non realistici

missing_data_cities = df_cities[df_cities['AverageTemperature'].isnull()]
missing_data_cities

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
35039,1884-09-01,NaN,NaN,Abengourou,Côte D'Ivoire,7.23N,4.05W
35040,1884-10-01,NaN,NaN,Abengourou,Côte D'Ivoire,7.23N,4.05W
35041,1884-11-01,NaN,NaN,Abengourou,Côte D'Ivoire,7.23N,4.05W
35042,1884-12-01,NaN,NaN,Abengourou,Côte D'Ivoire,7.23N,4.05W
35043,1885-01-01,NaN,NaN,Abengourou,Côte D'Ivoire,7.23N,4.05W
...,...,...,...,...,...,...,...
8574610,1885-06-01,NaN,NaN,Zinder,Niger,13.66N,9.08E
8574611,1885-07-01,NaN,NaN,Zinder,Niger,13.66N,9.08E
8574612,1885-08-01,NaN,NaN,Zinder,Niger,13.66N,9.08E
8574613,1885-09-01,NaN,NaN,Zinder,Niger,13.66N,9.08E


In [79]:
# dopo aver esplorato il dataset dei dati mancanti si decide di elimanare tutte le osservazioni antecedenti al 1894-01-01

df_cities= df_cities[df_cities['dt']>='1894-01-01']

In [80]:
# le misurazioni mancanti sono 45 in totale e le città non hanno più di due misurazioni vicine nulle

missing_data_cities_1 = df_cities[df_cities['AverageTemperature'].isnull()]
missing_data_cities_1.shape

(45, 7)

In [81]:
# si procede tramite interpolazione per inferire i dati mancanti

df_cities=df_cities.interpolate()

In [4]:
# lista nazioni in ordine di ordinamento per dt, City e Longitude 

country_city_list=list(df_cities.sort_values(['dt','City','Longitude'])[:3510]['Country'])

In [86]:
# si importa il dataset aggiornato con le misurazioni per ogni città fino al 2020-12-01

df_agg= pd.read_csv('Data/dataset_citta_aggiornamento_2020.csv')

In [87]:
df_agg

,Unnamed: 0,dt,AverageTemperature,City,Latitude,Longitude
0,0,2013-01-01,-0.390,Århus,57.05N,10.33E
1,1,2013-02-01,-0.403,Århus,57.05N,10.33E
2,2,2013-03-01,-0.468,Århus,57.05N,10.33E
3,3,2013-04-01,5.393,Århus,57.05N,10.33E
4,4,2013-05-01,12.744,Århus,57.05N,10.33E
...,...,...,...,...,...,...
336955,91,2020-08-01,27.905,Palmas,10.45S,48.27W
336956,92,2020-09-01,29.548,Palmas,10.45S,48.27W
336957,93,2020-10-01,29.012,Palmas,10.45S,48.27W
336958,94,2020-11-01,27.360,Palmas,10.45S,48.27W


In [88]:
# si sistemano alcuni errori nel dataset dell'aggiornament nella colonna 'dt' tramite sostituzione manuale 

df_agg['dt']=df_agg['dt'].replace(['11-12-01','6-07-01','8-09-01'],['2014-12-01', '2014-07-01', '2016-09-01'] )

In [89]:
# si resetta l'indice per entrambi i dataset per renderli compatibili

df_cities=df_cities.reset_index()[['dt', 'AverageTemperature','City', 'Latitude', 'Longitude']]
df_agg=df_agg.reset_index()[['dt', 'AverageTemperature','City', 'Latitude', 'Longitude']]

In [90]:
# si uniscono i due dataset

frames = [df_cities,df_agg]
df_cities_new = pd.concat(frames)

In [91]:
df_cities_new= df_cities_new.sort_values(by=['dt','City','Longitude'])
df_cities_new= df_cities_new.reset_index()[['dt', 'AverageTemperature','City', 'Latitude', 'Longitude']]
df_cities_new

,dt,AverageTemperature,City,Latitude,Longitude
0,1894-01-01,7.475,A Coruña,42.59N,8.73W
1,1894-01-01,-0.244,Aachen,50.63N,6.34E
2,1894-01-01,0.250,Aalborg,57.05N,10.33E
3,1894-01-01,25.866,Aba,5.63N,8.07E
4,1894-01-01,11.048,Abadan,29.74N,48.00E
...,...,...,...,...,...
5349235,2020-12-01,5.009,Århus,57.05N,10.33E
5349236,2020-12-01,9.843,Çorlu,40.99N,27.69E
5349237,2020-12-01,4.694,Çorum,40.99N,34.08E
5349238,2020-12-01,-13.483,Öskemen,50.63N,82.39E


In [92]:
# inserisco la colonna delle nazioni

country_column=country_city_list*1524
df_cities_new['Country']=country_column
df_cities_new

,dt,AverageTemperature,City,Latitude,Longitude,Country
0,1894-01-01,7.475,A Coruña,42.59N,8.73W,Spain
1,1894-01-01,-0.244,Aachen,50.63N,6.34E,Germany
2,1894-01-01,0.250,Aalborg,57.05N,10.33E,Denmark
3,1894-01-01,25.866,Aba,5.63N,8.07E,Nigeria
4,1894-01-01,11.048,Abadan,29.74N,48.00E,Iran
...,...,...,...,...,...,...
5349235,2020-12-01,5.009,Århus,57.05N,10.33E,Denmark
5349236,2020-12-01,9.843,Çorlu,40.99N,27.69E,Turkey
5349237,2020-12-01,4.694,Çorum,40.99N,34.08E,Turkey
5349238,2020-12-01,-13.483,Öskemen,50.63N,82.39E,Kazakhstan


In [98]:
# raccolta in lista delle coordinate di ogni città

lat=[]
lon=[]
df_cities_ll= df_cities_new[:3510]
for i in range(df_cities_ll.shape[0]):
    lat.append(df_cities_new.loc[i]['Latitude'])
    lon.append(df_cities_new.loc[i]['Longitude'])

In [99]:
# trasformazione delle coordinate sostituendo la sigla cardinale con il segno positivo o negativo e 
# trasformando la stringa in float

lat_new=[]

for c in lat:
    NS= c[-1]
    l_float= float(c[:-1])
    if NS== 'S':
        lat1= l_float - 2*l_float
        lat_new.append(lat1)
    else:
        lat1= l_float
        lat_new.append(lat1)

lon_new=[]

for c in lon:
    EW= c[-1]
    lo_float= float(c[:-1])
    if EW== 'W':
        lon1= lo_float - 2*lo_float
        lon_new.append(lon1)
    else:
        lon1= lo_float
        lon_new.append(lon1)

In [100]:
# aggiunta delle nuove coordinate nel df (1524 è il numero di misurazioni per ogni città)

lat_new1= lat_new*1524
lon_new1= lon_new*1524
df_cities_new['lat_new']= lat_new1
df_cities_new['lon_new']= lon_new1

df_cities_new

,dt,AverageTemperature,City,Latitude,Longitude,Country,lat_new,lon_new
0,1894-01-01,7.475,A Coruña,42.59N,8.73W,Spain,42.59,-8.73
1,1894-01-01,-0.244,Aachen,50.63N,6.34E,Germany,50.63,6.34
2,1894-01-01,0.250,Aalborg,57.05N,10.33E,Denmark,57.05,10.33
3,1894-01-01,25.866,Aba,5.63N,8.07E,Nigeria,5.63,8.07
4,1894-01-01,11.048,Abadan,29.74N,48.00E,Iran,29.74,48.00
...,...,...,...,...,...,...,...,...
5349235,2020-12-01,5.009,Århus,57.05N,10.33E,Denmark,57.05,10.33
5349236,2020-12-01,9.843,Çorlu,40.99N,27.69E,Turkey,40.99,27.69
5349237,2020-12-01,4.694,Çorum,40.99N,34.08E,Turkey,40.99,34.08
5349238,2020-12-01,-13.483,Öskemen,50.63N,82.39E,Kazakhstan,50.63,82.39


In [101]:
# creazione di un codice univoco (cll) per ogni città dato dalla combinazione di nome città, latitudine e longitudine

cll=[]
df_cities_cll=df_cities_new[:3510]

for i in range(df_cities_cll.shape[0]):
    cll.append(df_cities_new.loc[i]['City']+' '+str(df_cities_new.loc[i]['lat_new'])+' '+str(df_cities_new.loc[i]['lon_new']))


In [102]:
# aggiunta del codice univoco al dataset

cll_1=cll*1524
df_cities_new['CLL']= cll_1
df_cities_new

,dt,AverageTemperature,City,Latitude,Longitude,Country,lat_new,lon_new,CLL
0,1894-01-01,7.475,A Coruña,42.59N,8.73W,Spain,42.59,-8.73,A Coruña 42.59 -8.73
1,1894-01-01,-0.244,Aachen,50.63N,6.34E,Germany,50.63,6.34,Aachen 50.63 6.34
2,1894-01-01,0.250,Aalborg,57.05N,10.33E,Denmark,57.05,10.33,Aalborg 57.05 10.33
3,1894-01-01,25.866,Aba,5.63N,8.07E,Nigeria,5.63,8.07,Aba 5.63 8.07
4,1894-01-01,11.048,Abadan,29.74N,48.00E,Iran,29.74,48.00,Abadan 29.74 48.0
...,...,...,...,...,...,...,...,...,...
5349235,2020-12-01,5.009,Århus,57.05N,10.33E,Denmark,57.05,10.33,Århus 57.05 10.33
5349236,2020-12-01,9.843,Çorlu,40.99N,27.69E,Turkey,40.99,27.69,Çorlu 40.99 27.69
5349237,2020-12-01,4.694,Çorum,40.99N,34.08E,Turkey,40.99,34.08,Çorum 40.99 34.08
5349238,2020-12-01,-13.483,Öskemen,50.63N,82.39E,Kazakhstan,50.63,82.39,Öskemen 50.63 82.39


In [103]:
# eliminizione colonne superflue e rinominazione di alcune colonne

df_cities_new=df_cities_new[['dt','AverageTemperature', 'City', 'Country', 'lat_new', 'lon_new','CLL']]
df_cities_new=df_cities_new.rename(columns={'lat_new':'Latitude','lon_new':'Longitude'})
df_cities_new

,dt,AverageTemperature,City,Country,Latitude,Longitude,CLL
0,1894-01-01,7.475,A Coruña,Spain,42.59,-8.73,A Coruña 42.59 -8.73
1,1894-01-01,-0.244,Aachen,Germany,50.63,6.34,Aachen 50.63 6.34
2,1894-01-01,0.250,Aalborg,Denmark,57.05,10.33,Aalborg 57.05 10.33
3,1894-01-01,25.866,Aba,Nigeria,5.63,8.07,Aba 5.63 8.07
4,1894-01-01,11.048,Abadan,Iran,29.74,48.00,Abadan 29.74 48.0
...,...,...,...,...,...,...,...
5349235,2020-12-01,5.009,Århus,Denmark,57.05,10.33,Århus 57.05 10.33
5349236,2020-12-01,9.843,Çorlu,Turkey,40.99,27.69,Çorlu 40.99 27.69
5349237,2020-12-01,4.694,Çorum,Turkey,40.99,34.08,Çorum 40.99 34.08
5349238,2020-12-01,-13.483,Öskemen,Kazakhstan,50.63,82.39,Öskemen 50.63 82.39


In [104]:
# salvataggio del dataset

df_cities_new.to_csv('Data/dataset_cities_finale_2020.csv')

# Preparazione dati nazioni per la visualizzazione grafica 

In [2]:
df_countries=pd.read_csv('Data/GlobalLandTemperaturesByCountry.csv')

In [286]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [287]:
# uniformo nomenclatura delle nazioni per far si che corrispondano a quelle presenti nel dataframe df_countries

correction_dict={'S. Sudan': 'Sudan', 'Timor-Leste': 'Timor Leste', 'Dominican Rep.': 'Dominican Republic', 'Guinea-Bissau':'Guinea Bissau',
                 'Dem. Rep. Congo' : 'Congo', 'Eq. Guinea' : 'Equatorial Guinea', 'United States of America':'United States',
                'Czechia' : 'Czech Republic', 'Central African Republicblic' : 'Central African Republic', 'W. Sahara':'Western Sahara',
                'Bosnia and Herz.' :'Bosnia And Herzegovina', 'N. Cyprus':'Cyprus', 'Palestine':'Palestina',
                 'Trinidad and Tobago':'Trinidad And Tobago', 'eSwatini' : 'Swaziland', 'Somaliland' : 'Somalia',
                 'Solomon Is.' : 'Solomon Islands', str("Côte d'Ivoire") : str("Côte D'Ivoire"), 'Myanmar' : 'Burma', 
                 'Brunei' : 'Malaysia', 'Kosovo':'Serbia','Central African Rep.':'Central African Republic', 'Denmark': 'Denmark (Europe)'}

In [288]:
world['name']=world['name'].replace(correction_dict)

In [289]:
# elimino nazioni su cui non abbiamo dati

world=world[(world['name']!='Falkland Is.') &
            (world['name']!='Fr. S. Antarctic Lands') & 
            (world['name']!='Vanuatu')&(world['name']!='Antarctica') & 
            (world['continent']!='Seven seas (open ocean)')].reset_index()[['pop_est','continent','name','iso_a3','gdp_md_est','geometry']]

In [7]:
# insieme delle countries che mi servono per la visualizzazione

country_set=set(world['name'])

In [8]:
df_countries

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland
...,...,...,...,...
577457,2013-05-01,19.059,1.022,Zimbabwe
577458,2013-06-01,17.613,0.473,Zimbabwe
577459,2013-07-01,17.000,0.453,Zimbabwe
577460,2013-08-01,19.759,0.717,Zimbabwe


In [9]:
# Per i nostri fini la colonna AverageTemperatureUncertainty non ci serve

df_countries.drop(axis=0,columns={'AverageTemperatureUncertainty'},inplace=True)

In [10]:
# costruisco dataframe per la visualizzazione con le nazioni in country_set

frames=[]

for country in country_set:
    
    df_c=df_countries[df_countries['Country']==country]
    
    frames.append(df_c)
    
df_countries_viz=pd.concat(frames)
    

In [11]:
#riordino indici e sort_values per Country

df_countries_viz=df_countries_viz.sort_values(['Country','dt']).reset_index()[['dt','AverageTemperature','Country']]

In [12]:
df_countries_viz

,dt,AverageTemperature,Country
0,1838-04-01,13.008,Afghanistan
1,1838-05-01,NaN,Afghanistan
2,1838-06-01,23.950,Afghanistan
3,1838-07-01,26.877,Afghanistan
4,1838-08-01,24.938,Afghanistan
...,...,...,...
402550,2013-05-01,19.059,Zimbabwe
402551,2013-06-01,17.613,Zimbabwe
402552,2013-07-01,17.000,Zimbabwe
402553,2013-08-01,19.759,Zimbabwe


In [13]:
# Controllo le prime occorrenze dei misuramenti per ogni nazione

first_occurencies={}

for i in range(1,df_countries_viz.shape[0]):
    
    if df_countries_viz.loc[i-1]['Country'] != df_countries_viz.loc[i]['Country']:
        
        first_occurencies[df_countries_viz.loc[i]['Country']] = df_countries_viz.loc[i]['dt']

In [14]:
max(first_occurencies.values())

'1882-01-01'

In [15]:
# filtro dataframe per 1882 < dt < 2012

df_countries_viz=df_countries_viz[(df_countries_viz['dt'] >= '1882-01-01') & (df_countries_viz['dt'] <= '2012-12-01')]

In [16]:
df_countries_viz=df_countries_viz.reset_index()[['dt','AverageTemperature','Country']]
df_countries_viz

,dt,AverageTemperature,Country
0,1882-01-01,0.525,Afghanistan
1,1882-02-01,0.475,Afghanistan
2,1882-03-01,6.989,Afghanistan
3,1882-04-01,13.606,Afghanistan
4,1882-05-01,20.302,Afghanistan
...,...,...,...
262519,2012-08-01,18.946,Zimbabwe
262520,2012-09-01,22.609,Zimbabwe
262521,2012-10-01,23.482,Zimbabwe
262522,2012-11-01,24.606,Zimbabwe


In [17]:
# carico dataset aggiornamenti 

df_countries_update=pd.read_csv('Data/aggiornamento_dati_nazioni_1.csv')
df_countries_update.drop(axis=1,columns='Unnamed: 0',inplace=True)

In [18]:
# unisco i dataframe 

f=[df_countries_viz,df_countries_update]

df_countries_viz_updated=pd.concat(f)

In [19]:
df_countries_viz_updated=df_countries_viz_updated.sort_values(['Country','dt']).reset_index()[['dt','AverageTemperature','Country']]
df_countries_viz_updated

,dt,AverageTemperature,Country
0,1882-01-01,0.525,Afghanistan
1,1882-02-01,0.475,Afghanistan
2,1882-03-01,6.989,Afghanistan
3,1882-04-01,13.606,Afghanistan
4,1882-05-01,20.302,Afghanistan
...,...,...,...
277795,2020-08-01,19.621,Zimbabwe
277796,2020-09-01,22.638,Zimbabwe
277797,2020-10-01,24.588,Zimbabwe
277798,2020-11-01,25.934,Zimbabwe


In [20]:
# cerco eventuali valori NaN nelle misurazioni delle temperature

nan_list=[]
null_df=df_countries_viz_updated.isnull()['AverageTemperature']

for i in range(df_countries_viz_updated.shape[0]):

    if null_df.loc[i]:
        
        nan_list.append([i,df_countries_viz_updated.loc[i]['Country'],df_countries_viz_updated.loc[i]['dt']])
        
        

In [5]:
nan_list

In [22]:
# filtro dal 1890 in su per eliminare i nan

df_countries_viz_updated=df_countries_viz_updated[df_countries_viz_updated['dt'] >= '1891-01-01']

In [23]:
# applico interpolate per eliminare gli ultimi nan in Perù in quanto isolati

df_countries_viz_updated=df_countries_viz_updated.interpolate()

In [24]:
df_countries_viz_updated=df_countries_viz_updated.reset_index()[['dt','AverageTemperature','Country']]

In [25]:
df_countries_viz_updated['Country']=df_countries_viz_updated['Country'].replace('Denmark (Europe)','Denmark')

In [26]:
# affianco colonna degli anni

year_list=[]

for i in range(df_countries_viz_updated.shape[0]):
    
    year_list.append(df_countries_viz_updated.loc[i]['dt'][:4])
    
    


In [27]:
df_countries_viz_updated['Year']=year_list

In [6]:
country_list=set(df_countries_viz_updated['Country'])

In [28]:
df_mean=df_countries_viz_updated.groupby(['Country','Year']).mean()

In [32]:
df_mean

AverageTemperature
Country     Year                    
Afghanistan 1891           13.237250
            1892           14.076000
            1893           13.436750
            1894           13.466750
            1895           13.981750
...                              ...
Zimbabwe    2016           22.668333
            2017           21.590083
            2018           22.140000
            2019           22.698167
            2020           21.959250

[21654 rows x 1 columns]

In [36]:
# ricavo le medie di riferimento 1951-1980

reference_temp_dict={}

for country in country_list:
    
    s=0
    
    df_country=df_mean.loc[country]
    
    for y in range(1951,1981):
        
        s+= df_country.loc[str(y)]['AverageTemperature']
        
    reference_temp_dict[country]=s/len(list(range(1951,1981)))  

In [41]:
#costruisco dataframe delle anomalie 

y=[]
anom=[]
c=[]

for country in country_list:
    
    df_c=df_mean.loc[country].reset_index()
    
    for i in range(df_c.shape[0]):
        
        c.append(country)
        y.append(df_c.loc[i]['Year'])
        anom.append(df_c.loc[i]['AverageTemperature']-reference_temp_dict[country])
        
anom_dataset_dict={'Year':y,'Country':c,'Anomaly':anom}

df_anomalies_countries=pd.DataFrame.from_dict(anom_dataset_dict)

In [42]:
df_anomalies_countries=df_anomalies_countries.sort_values(['Country','Year']).reset_index()[['Year','Country','Anomaly']]
df_anomalies_countries

,Year,Country,Anomaly
0,1891,Afghanistan,-0.823597
1,1892,Afghanistan,0.015153
2,1893,Afghanistan,-0.624097
3,1894,Afghanistan,-0.594097
4,1895,Afghanistan,-0.079097
...,...,...,...
21649,2016,Zimbabwe,1.498344
21650,2017,Zimbabwe,0.420094
21651,2018,Zimbabwe,0.970011
21652,2019,Zimbabwe,1.528178


In [37]:
# controllo last occurencies 

last_occurencies={}

for i in range(1,df_countries_viz_updated.shape[0]):
    
    if df_countries_viz_updated.loc[i-1]['Country'] != df_countries_viz_updated.loc[i]['Country']:
        
        last_occurencies[df_countries_viz_updated.loc[i-1]['Country']] = df_countries_viz_updated.loc[i-1]['dt']
        

In [69]:
missing_data={}

for k,v  in last_occurencies.items():
    
    if v<'2020-12-01':
        
        missing_data[k]=v

In [70]:
missing_data

{'Bahamas': '2016-05-01',
 'Burma': '2016-05-01',
 'Congo': '2016-05-01',
 'Czech Republic': '2016-05-01',
 "Côte D'Ivoire": '2012-12-01',
 'Greenland': '2016-05-01',
 'New Caledonia': '2016-05-01',
 'Palestina': '2016-05-01',
 'Serbia': '2016-05-01',
 'Swaziland': '2016-05-01',
 'Tanzania': '2016-05-01',
 'Timor Leste': '2016-05-01',
 'United States': '2016-05-01'}

## Occorre stimare le temperature fino al 2020 per le nazioni in cui mancano i misuramenti verranno stimati calcolando una media attraverso le città di quelle nazioni

In [105]:
df_cities_new

,dt,AverageTemperature,City,Country,Latitude,Longitude,CLL
0,1894-01-01,7.475,A Coruña,Spain,42.59,-8.73,A Coruña 42.59 -8.73
1,1894-01-01,-0.244,Aachen,Germany,50.63,6.34,Aachen 50.63 6.34
2,1894-01-01,0.250,Aalborg,Denmark,57.05,10.33,Aalborg 57.05 10.33
3,1894-01-01,25.866,Aba,Nigeria,5.63,8.07,Aba 5.63 8.07
4,1894-01-01,11.048,Abadan,Iran,29.74,48.00,Abadan 29.74 48.0
...,...,...,...,...,...,...,...
5349235,2020-12-01,5.009,Århus,Denmark,57.05,10.33,Århus 57.05 10.33
5349236,2020-12-01,9.843,Çorlu,Turkey,40.99,27.69,Çorlu 40.99 27.69
5349237,2020-12-01,4.694,Çorum,Turkey,40.99,34.08,Çorum 40.99 34.08
5349238,2020-12-01,-13.483,Öskemen,Kazakhstan,50.63,82.39,Öskemen 50.63 82.39


In [109]:
df_cities_new[df_cities_new['Country']=='United States'].groupby('dt').mean().reset_index()

,dt,AverageTemperature,Latitude,Longitude
0,1894-01-01,4.461938,36.592218,-98.177549
1,1894-02-01,3.634872,36.592218,-98.177549
2,1894-03-01,9.956202,36.592218,-98.177549
3,1894-04-01,13.735401,36.592218,-98.177549
4,1894-05-01,17.738961,36.592218,-98.177549
...,...,...,...,...
1519,2020-08-01,25.506401,36.592218,-98.177549
1520,2020-09-01,21.913926,36.592218,-98.177549
1521,2020-10-01,16.885984,36.592218,-98.177549
1522,2020-11-01,12.303626,36.592218,-98.177549


In [133]:
# definisco una funzione per calcolare un approssimazione dei dati mancanti

def c_year_approx(country):
    
    y=[]
    avg_t=[]
    c=[]
    j=0
    df_c=df_cities_new[df_cities_new['Country']==country].groupby('dt').mean().reset_index()
    
    for i in range(df_c.shape[0]):
        
        y.append(df_c.loc[i]['dt'][:4])
        
    df_c['Year']=y
        
    df_c=df_c.groupby('Year').mean()[['AverageTemperature']]
        
    df_c['AnomalyApprox']=df_c['AverageTemperature']-reference_temp_dict[country]
    
    return df_c
     
    

In [134]:
c_year_approx('United States')

,AverageTemperature,AnomalyApprox
Year,,
1894,14.101181,5.307853
1895,13.554607,4.761279
1896,14.372444,5.579116
1897,14.063780,5.270452
1898,14.074488,5.281160
...,...,...
2016,15.970165,7.176838
2017,15.928038,7.134710
2018,15.476891,6.683564


In [139]:
# calcolo il discostamento medio delle anomalie per ottenere una loro approssimazione da utilizzare per la visualizzazione
# gli utimi misuramenti inoltre sono sfasati, li elimino

def anom_approx(df_c,country):
    
    var_approx=0
    
    df_anom_c=df_anomalies_countries[df_anomalies_countries['Country']=='United States']
    df_anom_c.drop(df_anom_c.tail(1).index,inplace=True)
    df_anom_c=df_anom_c.tail(50).reset_index()
    
    df_c=df_c[:missing_data[country][:4]]
    
    df_c=df_c.tail(50).reset_index()
    
    df_c['Var']=df_c['AnomalyApprox']-df_anom_c['Anomaly']
    
    var_list=list(df_c['Var'])
    
    for v in var_list:
        
        var_approx+= v
        
    var_approx=var_approx/len(var_list)
    
    return var_approx

In [140]:
anom_approx(c_year_approx('United States'),'United States')

5.554902684824901

In [189]:
df_anom_countries=df_anomalies_countries

In [ ]:
# stima delle anomalie
err=[]

for country in missing_data.keys():
    
    cc=[]
    anom=[]
    year=[]
    
    try:
          
        df_anom_countries=df_anom_countries.drop(df_anom_countries[df_anom_countries['Country']==country].tail(1).index)
        f=[df_anom_countries]
        df_c=c_year_approx(country)
        v_approx=anom_approx(df_c,country)
        df_c['Anomaly']=df_c['AnomalyApprox']-v_approx

        for y in range(int(missing_data[country][:4]),2021):

            cc.append(country)
            anom.append(df_c.loc[str(y)]['Anomaly'])
            year.append(str(y))


        missing_country_anom_approx={'Year':year,'Country':cc,'Anomaly':anom}

        df_missing_country=pd.DataFrame.from_dict(missing_country_anom_approx)

        f.append(df_missing_country)    
        
        df_anom_countries=pd.concat(f)

    except :
        
        err.append(country)    

In [191]:
df_anom_countries=df_anom_countries.sort_values(['Country','Year']).reset_index()[['Year','Country','Anomaly']]

In [192]:
df_anom_countries[df_anom_countries['Country']=='United States'].tail(10)

,Year,Country,Anomaly
20640,2011,United States,0.756589
20641,2012,United States,1.467756
20642,2013,United States,0.465089
20643,2014,United States,0.722422
20644,2015,United States,1.536089
20645,2016,United States,1.621935
20646,2017,United States,1.579807
20647,2018,United States,1.128661
20648,2019,United States,0.791039
20649,2020,United States,1.503988


In [193]:
# creazione dataframe anomalie globali

avg_global_anomaly=df_anom_countries.groupby('Year').mean().reset_index()
avg_global_anomaly

,Year,Anomaly
0,1891,-0.634047
1,1892,-0.585573
2,1893,-0.708100
3,1894,-0.559085
4,1895,-0.518558
...,...,...
125,2016,1.252929
126,2017,1.099427
127,2018,1.177114
128,2019,1.336224


## Non abbiamo dati per poter aggiornare i misuramenti per le seguenti nazioni

In [163]:
err

['Greenland', 'New Caledonia', 'Palestina', 'Timor Leste']

## I dati mancanti verranno approssimati interpolando linearmente l'ultimo misuramento a disposizione con il valore dell'anomalia media globale nel 2020 (1.3°)

In [194]:
for country in err:
    
    c=[]
    year=[]
    anom=[]
    
    #df_anom_countries=df_anom_countries.drop(df_anom_countries[df_anom_countries['Country']==country].tail(1).index)
    
    f=[df_anom_countries]
    
    for y in range(int(missing_data[country][:4]),2021):
        
        if y == 2020:
            
            c.append(country)
            year.append(str(y))
            anom.append(1.3)
            
        else:
            
            c.append(country)
            year.append(str(y))
            anom.append(np.nan)
            
    missing_val_dict={'Year': year, 'Country': c, 'Anomaly':anom}
            
    missing_val_df=pd.DataFrame.from_dict(missing_val_dict)
    
    f.append(missing_val_df)
    
    df_anom_countries=pd.concat(f)
            
df_anom_countries=df_anom_countries.sort_values(['Country','Year'])   

In [205]:
df_anom_countries[df_anom_countries['Country']=='Timor Leste'].tail(10)

,Year,Country,Anomaly
19345,2011,Timor Leste,-0.153842
19346,2012,Timor Leste,0.031908
19347,2013,Timor Leste,0.464575
19348,2014,Timor Leste,0.298408
19349,2015,Timor Leste,0.260992
0,2016,Timor Leste,NaN
1,2017,Timor Leste,NaN
2,2018,Timor Leste,NaN
3,2019,Timor Leste,NaN
4,2020,Timor Leste,1.300000


In [200]:
# sistemazione per la visualizzazione, valore anomalo in greenland

df_anom_countries.at[7664,'Anomaly'] = 0.883400

In [207]:
df_anom_countries=df_anom_countries.reset_index()[['Year','Country','Anomaly']]
df_anom_countries=df_anom_countries.interpolate()
df_anom_countries

,Year,Country,Anomaly
0,1891,Afghanistan,-0.823597
1,1892,Afghanistan,0.015153
2,1893,Afghanistan,-0.624097
3,1894,Afghanistan,-0.594097
4,1895,Afghanistan,-0.079097
...,...,...,...
21705,2016,Zimbabwe,1.498344
21706,2017,Zimbabwe,0.420094
21707,2018,Zimbabwe,0.970011
21708,2019,Zimbabwe,1.528178


In [208]:
# salvo dataset anomalie nazioni

df_anom_countries.to_csv('Data/dataset_visualiz_anom_2020.csv')

In [209]:
#salvo dataset anomalia media globale

avg_global_anomaly.to_csv('Data/dataset_visualiz_anom_glob_2020.csv')

# Costruzione del dataset coordinate-anomalie delle città per evidenziare le città in cui si registra un'alta variazione di temperatura

In [210]:
df_cities_new

,dt,AverageTemperature,City,Country,Latitude,Longitude,CLL
0,1894-01-01,7.475,A Coruña,Spain,42.59,-8.73,A Coruña 42.59 -8.73
1,1894-01-01,-0.244,Aachen,Germany,50.63,6.34,Aachen 50.63 6.34
2,1894-01-01,0.250,Aalborg,Denmark,57.05,10.33,Aalborg 57.05 10.33
3,1894-01-01,25.866,Aba,Nigeria,5.63,8.07,Aba 5.63 8.07
4,1894-01-01,11.048,Abadan,Iran,29.74,48.00,Abadan 29.74 48.0
...,...,...,...,...,...,...,...
5349235,2020-12-01,5.009,Århus,Denmark,57.05,10.33,Århus 57.05 10.33
5349236,2020-12-01,9.843,Çorlu,Turkey,40.99,27.69,Çorlu 40.99 27.69
5349237,2020-12-01,4.694,Çorum,Turkey,40.99,34.08,Çorum 40.99 34.08
5349238,2020-12-01,-13.483,Öskemen,Kazakhstan,50.63,82.39,Öskemen 50.63 82.39


In [214]:
# inserimento colonna anni
y_column=[]

for year in range(1894,2021):
    
    for i in range(12):
        
        for j in range(3510):
        
            y_column.append(year)

In [216]:
df_cities_new['Year']=y_column
df_cities_new

,dt,AverageTemperature,City,Country,Latitude,Longitude,CLL,Year
0,1894-01-01,7.475,A Coruña,Spain,42.59,-8.73,A Coruña 42.59 -8.73,1894
1,1894-01-01,-0.244,Aachen,Germany,50.63,6.34,Aachen 50.63 6.34,1894
2,1894-01-01,0.250,Aalborg,Denmark,57.05,10.33,Aalborg 57.05 10.33,1894
3,1894-01-01,25.866,Aba,Nigeria,5.63,8.07,Aba 5.63 8.07,1894
4,1894-01-01,11.048,Abadan,Iran,29.74,48.00,Abadan 29.74 48.0,1894
...,...,...,...,...,...,...,...,...
5349235,2020-12-01,5.009,Århus,Denmark,57.05,10.33,Århus 57.05 10.33,2020
5349236,2020-12-01,9.843,Çorlu,Turkey,40.99,27.69,Çorlu 40.99 27.69,2020
5349237,2020-12-01,4.694,Çorum,Turkey,40.99,34.08,Çorum 40.99 34.08,2020
5349238,2020-12-01,-13.483,Öskemen,Kazakhstan,50.63,82.39,Öskemen 50.63 82.39,2020


In [223]:
# raggruppamento per identificatore e per anno di ogni città
df_cities_group=df_cities_new.groupby(['CLL','Year']).mean()

In [219]:
cll_set=set(df_cities_new['CLL'])

In [242]:
#ricavo reference temp per ciascuna città

reference_temp_cities={}

for city in cll_set:
    
    ref_temp=0
    
    for year in range(1951,1981):
        
        ref_temp += df_cities_group.loc[city].loc[year]['AverageTemperature']
        
    reference_temp_cities[city] = ref_temp/len(list(range(1951,1981)))

In [7]:
reference_temp_cities

In [ ]:
# creazione dataframe coordinate, anomalie e città

llat=[]
llon=[]
y_list=[]
city_anom=[]
cll_list=[]


for city in cll_set:
    
    for year in range(1894,2021):
        
        y_list.append(year)
        
        city_anom.append(df_cities_group.loc[city].loc[year]['AverageTemperature']-reference_temp_cities[city])
        
        llat.append(float(city.split()[-2:][0]))
        llon.append(float(city.split()[-2:][1]))
        
        cll_list.append(city)
        
coord_anom_plot_dict={'Year':y_list,'Latitude':llat,'Longitude':llon, 'Anomaly':city_anom,'CLL':cll_list}

coord_anom_df=pd.DataFrame.from_dict(coord_anom_plot_dict)

In [302]:
coord_anom_df

,Year,Latitude,Longitude,Anomaly,CLL
0,1894,23.31,88.25,-0.387006,Panihati 23.31 88.25
1,1895,23.31,88.25,-0.538339,Panihati 23.31 88.25
2,1896,23.31,88.25,0.020661,Panihati 23.31 88.25
3,1897,23.31,88.25,-0.053006,Panihati 23.31 88.25
4,1898,23.31,88.25,-0.461672,Panihati 23.31 88.25
...,...,...,...,...,...
445765,2016,23.31,88.25,1.199994,Bhatpara 23.31 88.25
445766,2017,23.31,88.25,0.830994,Bhatpara 23.31 88.25
445767,2018,23.31,88.25,0.563578,Bhatpara 23.31 88.25
445768,2019,23.31,88.25,0.702828,Bhatpara 23.31 88.25


In [269]:
coord_anom_df.to_csv('Data/city_anom_plot.csv')

# Preparo geodataframe per la visualizzazione inserendo per ogni anno colonna anomalie

In [323]:
world['name']=world['name'].replace('Denmark (Europe)','Denmark')

In [325]:
# inserisco colonna anomalie per ciascun anno

for h in range(1891,2021):
    
    print(h, end=' - ')

    a={}

    anomalies=[]

    for i in range(df_anom_countries.shape[0]):

        if df_anom_countries.loc[i]['Year']==str(h):

            a[df_anom_countries.loc[i]['Country']]=(df_anom_countries.loc[i]['Anomaly'])

    anomalies=list(range(world.shape[0]))

    for k, v in a.items():

        for i in range(world.shape[0]):

            if world.loc[i]['name'] == k:

                anomalies[i] = v


    world[str(h)] = anomalies

1891 - 1892 - 1893 - 1894 - 1895 - 1896 - 1897 - 1898 - 1899 - 1900 - 1901 - 1902 - 1903 - 1904 - 1905 - 1906 - 1907 - 1908 - 1909 - 1910 - 1911 - 1912 - 1913 - 1914 - 1915 - 1916 - 1917 - 1918 - 1919 - 1920 - 1921 - 1922 - 1923 - 1924 - 1925 - 1926 - 1927 - 1928 - 1929 - 1930 - 1931 - 1932 - 1933 - 1934 - 1935 - 1936 - 1937 - 1938 - 1939 - 1940 - 1941 - 1942 - 1943 - 1944 - 1945 - 1946 - 1947 - 1948 - 1949 - 1950 - 1951 - 1952 - 1953 - 1954 - 1955 - 1956 - 1957 - 1958 - 1959 - 1960 - 1961 - 1962 - 1963 - 1964 - 1965 - 1966 - 1967 - 1968 - 1969 - 1970 - 1971 - 1972 - 1973 - 1974 - 1975 - 1976 - 1977 - 1978 - 1979 - 1980 - 1981 - 1982 - 1983 - 1984 - 1985 - 1986 - 1987 - 1988 - 1989 - 1990 - 1991 - 1992 - 1993 - 1994 - 1995 - 1996 - 1997 - 1998 - 1999 - 2000 - 2001 - 2002 - 2003 - 2004 - 2005 - 2006 - 2007 - 2008 - 2009 - 2010 - 2011 - 2012 - 2013 - 2014 - 2015 - 2016 - 2017 - 2018 - 2019 - 2020 - 

In [326]:
world.to_file('world_viz_plot')